In [6]:
import json
import pandas as pd
from pathlib import Path
from typing import Dict, Any

# 指定目录和 json 文件
data_dir = Path('.')
json_files = [
    'refactored_alpha_gfn_bitseq.json',
    'rebuttal_bit_fl.json',
    'refactored_alpha_gfn_bitseq_icml2026.json'
]

def flatten_dict(d: Dict[str, Any], parent_key: str = '', sep: str = '_') -> Dict[str, Any]:
    """递归展平嵌套字典"""
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

# 加载所有 json 文件并解析为 DataFrame
all_data = []

for file in json_files:
    file_path = data_dir / file
    with open(file_path, 'r') as f:
        data = json.load(f)

    # 每个 json 文件的顶层 key 是实验名称，value 是参数字典
    for exp_name, exp_params in data.items():
        flat_params = flatten_dict(exp_params)
        flat_params['experiment_name'] = exp_name  # 保存实验名称
        flat_params['source_file'] = file  # 保存来源文件
        all_data.append(flat_params)

# 转换为 DataFrame
df = pd.DataFrame(all_data)

print(f"✓ 加载完成！共 {len(df)} 行")
print(f"✓ 列数: {len(df.columns)}")
print(f"\n数据源: {json_files}")
print(f"\nDataFrame 形状: {df.shape}")
print(f"\n前几行:")
df.head()

✓ 加载完成！共 1125 行
✓ 列数: 90

数据源: ['refactored_alpha_gfn_bitseq.json', 'rebuttal_bit_fl.json', 'refactored_alpha_gfn_bitseq_icml2026.json']

DataFrame 形状: (1125, 90)

前几行:


,M_size,_runtime,_step,_test_tokens__type,_timestamp,alpha,alpha_init,alpha_sched,alpha_warm_frac,avg_current_reward_eval,...,qgfn_tau,qgfn_variant,spearman_corr_test_unscaled,top_1000_count,top_100_count,top_k,avg_current_reward_eval_scaled,checkpoint_dir,checkpoint_every,resume_checkpoint
0,60.0,50082.229668,49999,histogram,1.758340e+09,0.503665,0.7,hold_exp,0.8,6.366812e-39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,60.0,90905.526636,49999,histogram,1.758381e+09,0.505497,0.8,hold_exp,0.8,1.149229e-36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,60.0,78799.941123,49999,histogram,1.758369e+09,0.494503,0.2,hold_exp,0.8,1.130852e-36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,60.0,31784.249645,49999,histogram,1.758290e+09,0.500000,0.5,hold_exp,0.8,4.573409e-34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,60.0,82301.522906,49999,histogram,1.758379e+09,0.498168,0.4,hold_exp,0.8,2.152297e-38,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
import re
import numpy as np

# 复制一份以免污染原始 df
df_all = df.copy()

# 解析实验名称中的参数
def parse_exp_name(exp_name: str) -> dict:
    params = {}
    ss_match = re.search(r'ss\((\d+)\)', exp_name)
    if ss_match:
        params['seed'] = int(ss_match.group(1))

    m_match = re.search(r'_m\(([^)]+)\)', exp_name)
    if m_match:
        params['method'] = m_match.group(1)

    k_match = re.search(r'k\(([^)]+)\)', exp_name)
    if k_match:
        params['k'] = k_match.group(1)

    a_match = re.search(r'_a\(([^)]+)\)', exp_name)
    if a_match:
        try:
            params['alpha_init_from_name'] = float(a_match.group(1))
        except ValueError:
            params['alpha_init_from_name'] = np.nan

    return params

parsed_rows = []
for _, row in df_all.iterrows():
    name = row.get('experiment_name', '')
    parsed = parse_exp_name(name)
    parsed_rows.append({
        **row,
        'seed': parsed.get('seed'),
        'method': parsed.get('method'),
        'k': parsed.get('k'),
        'alpha_init_name': parsed.get('alpha_init_from_name'),
        'alpha': row.get('alpha_init', row.get('alpha'))
    })

df_all = pd.DataFrame(parsed_rows)

print(f"总实验数: {len(df_all)}")
print(f"Unique seeds: {sorted(df_all['seed'].dropna().unique())}")
print(f"Unique k: {sorted(df_all['k'].dropna().unique())}")
print(f"Unique alphas (from alpha_init): {sorted(df_all['alpha'].dropna().unique())}")
print(f"Unique objectives/methods: {sorted(df_all['method'].dropna().unique())}")
print("\n数据来源分布:")
print(df_all.groupby(['source_file', 'method', 'k']).size())

# 只保留需要的指标
metrics = ['modes', 'spearman_corr_test']
available_metrics = [m for m in metrics if m in df_all.columns]
missing_metrics = [m for m in metrics if m not in df_all.columns]

if missing_metrics:
    print(f"⚠️ 缺失指标: {missing_metrics}")
    print(f"✓ 可用指标: {available_metrics}\n")
    metrics = available_metrics

objectives = sorted(df_all['method'].dropna().unique())
ks = sorted(df_all['k'].dropna().unique())

print(f"\n{'=' * 80}")
print(f"找到 {len(objectives)} 个 objectives: {objectives}")
print(f"找到 {len(ks)} 个 k: {ks}")
print(f"{'=' * 80}")

# 收集绘图配置
fig_configs = []

for obj in objectives:
    for k in ks:
        print(f"\n{'=' * 80}")
        print(f"Objective: {obj}, k: {k}")
        print(f"{'=' * 80}")

        df_obj = df_all[(df_all['method'] == obj) & (df_all['k'] == k)].copy()
        if len(df_obj) == 0:
            print("无数据，跳过")
            continue

        print(f"  实验数: {len(df_obj)}")
        print(f"  来源文件: {df_obj['source_file'].unique()}")

        df_averaged = df_obj.groupby('alpha')[metrics].mean().reset_index()
        df_averaged = df_averaged.sort_values('alpha')

        print("\n按 seed 平均后的结果 (基于 alpha_init):")
        print(df_averaged.to_string(index=False))

        print("\n最优 Alpha 分析:")
        print(f"{'-' * 80}")
        for metric in metrics:
            if len(df_averaged) > 0 and not df_averaged[metric].isna().all():
                best_idx = df_averaged[metric].idxmax()
                best_alpha = df_averaged.loc[best_idx, 'alpha']
                best_value = df_averaged.loc[best_idx, metric]
                print(f"  {metric:30s} -> 最优 alpha_init: {best_alpha:.4f}, 最优值: {best_value:.6f}")

        plot_metrics = [m for m in metrics if not df_averaged[m].isna().all()]
        if plot_metrics:
            fig_configs.append({
                'method': obj,
                'k': k,
                'data': df_averaged,
                'metrics': plot_metrics
            })

总实验数: 1125
Unique seeds: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4)]
Unique k: ['10', '2', '4', '6', '8']
Unique alphas (from alpha_init): [np.float64(0.1), np.float64(0.2), np.float64(0.3), np.float64(0.4), np.float64(0.5), np.float64(0.6), np.float64(0.7), np.float64(0.8), np.float64(0.9)]
Unique objectives/methods: ['db', 'subtb', 'tb']

数据来源分布:
source_file                                method  k 
rebuttal_bit_fl.json                       db      10    45
                                                   4     45
                                                   6     45
                                                   8     45
                                           subtb   10    45
                                                   4     45
                                                   6     45
                                                   8     45
refactored_alpha_gfn_bitseq.json           db      10    45
                                 

In [ ]:
import matplotlib.pyplot as plt

out_dir = Path('side_product')
out_dir.mkdir(exist_ok=True, parents=True)

print(f"\n{'=' * 80}\n生成图表\n{'=' * 80}")
for cfg in fig_configs:
    obj = cfg['method']
    k = cfg['k']
    data = cfg['data']
    metrics_to_plot = cfg['metrics']

    fig, axes = plt.subplots(len(metrics_to_plot), 1, figsize=(6, 3 * len(metrics_to_plot)))
    if len(metrics_to_plot) == 1:
        axes = [axes]

    for ax, metric in zip(axes, metrics_to_plot):
        ax.plot(data['alpha'], data[metric], marker='o')
        ax.set_title(f"{obj} | k={k} | metric={metric}")
        ax.set_xlabel('alpha_init')
        ax.set_ylabel(metric)
        ax.grid(True, linestyle='--', alpha=0.5)

    plt.tight_layout()
    save_path = out_dir / f"bit_analysis_{obj}_{k}.png"
    plt.savefig(save_path, dpi=200)
    plt.show()
    print(f"保存: {save_path}")

print("\n全部图表生成完毕。")